Zaimplementować najprostszą metodę Monte Carlo obliczania całki

$$I(f)=\int\limits_a^b f(x) dx$$

Przedstawić dwie implementację używającą modułu $\color{green}{numpy}$ oraz wykonać testy numeryczne.

Dla ustalonego $n\in\mathbb{N}$ metoda ta ma postać

$$MC_n(f) = \frac{b-a}{n}\sum\limits_{j=1}^n f(\xi_j), $$

gdzie $\xi_1,\xi_2,\ldots,\xi_n$ są niezależnymi zmiennymi losowymi o rozkładzie jednostajnym na $[a,b]$. Wariancja empiryczna ma postać

$$\bar V_n(f) = \frac{1}{n-1}\sum\limits_{j=1}^n\Bigl((b-a)f(\xi_j)-MC_n(f)\Bigr)^2$$
$$=\frac{1}{n-1}\sum\limits_{j=1}^n((b-a)f(\xi_j))^2-\frac{n}{n-1}(MC_n(f))^2$$

Empiryczne przedziały ufności na poziomie ufności $1-\alpha=0.95$, mają postać

$$\Bigl[MC_n(f)-2\cdot\sqrt{\frac{\bar V_n(f)}{n}}, MC_n(f)+2\cdot\sqrt{\frac{\bar V_n(f)}{n}}\Bigr]$$

In [1]:
import numpy as np

In [2]:
a = 0.0
b = np.pi

In [3]:
N = [11, 11**2, 11**3, 11**4]

In [4]:
I = 2.0+ (np.pi**3)/3

In [5]:
print(I)

12.335425560099939


In [6]:
def quad_MC(f, a, b, n):    
    random_samples_0 = np.random.uniform(a,b,n)
    f_values = f(random_samples_0)
    
    avg_f = np.mean(f_values)
    MC_f = (b-a) * avg_f
    
    random_samples_1 = np.power(f_values-avg_f,2)
    Var_f = (n / (n-1)) * np.power(b-a,2) * np.mean(random_samples_1)
    
    L_CI = MC_f - 2.0*np.sqrt(Var_f/n)
    R_CI = MC_f + 2.0*np.sqrt(Var_f/n)
    
    return MC_f, Var_f, L_CI, R_CI

In [7]:
def f_0(x):
    return np.sin(x)+x**2

In [8]:
N = [11, 11**2, 11**3, 11**4, 11**5, 11**7, 11**8]

In [9]:
%%time
for i in range(0, len(N)):
    Q, VAR, L_CI, R_CI = quad_MC(f_0, a, b, N[i])
    ERR = abs(I-Q)
    print("n = %.i, Quad MC= %.10f, wariancja = %.10f, [%.10f, %.10f], błąd = %.10f" % (N[i], Q, VAR, L_CI, R_CI, ERR) )

n = 11, Quad MC= 15.3349596505, wariancja = 120.9238167591, [8.7037985906, 21.9661207105], błąd = 2.9995340904
n = 121, Quad MC= 12.3014424313, wariancja = 76.3133083010, [10.7131244551, 13.8897604075], błąd = 0.0339831288
n = 1331, Quad MC= 11.9774414052, wariancja = 83.0531076614, [11.4778454523, 12.4770373582], błąd = 0.3579841549
n = 14641, Quad MC= 12.3387252072, wariancja = 82.2177567706, [12.1888508143, 12.4885996001], błąd = 0.0032996471
n = 161051, Quad MC= 12.3604504245, wariancja = 81.8264903939, [12.3153692477, 12.4055316014], błąd = 0.0250248644
n = 19487171, Quad MC= 12.3404401270, wariancja = 81.9405129695, [12.3363389838, 12.3445412702], błąd = 0.0050145669
n = 214358881, Quad MC= 12.3354025466, wariancja = 81.9285839765, [12.3341660954, 12.3366389978], błąd = 0.0000230135
Wall time: 11.8 s


$$2\cdot\int\limits_0^1 \frac{dx}{\sqrt{1-x^2}}=\pi$$

Uwaga: całka z funkcji z osobliwością w $x=1$, deterministyczne kwadratury mogą mieć problem. Funkcja nie jest całkowalna z kwadratem!

In [10]:
def g(x):
    return 2.0/np.sqrt(1-np.power(x,2))

In [11]:
quad_MC(g, 0.0, 1.0, 301111)

(3.154455940423833, 30.843828210883842, 3.1342140614971687, 3.1746978193504973)

In [12]:
%%time
for i in range(0, len(N)):
    Q, VAR, L_CI, R_CI = quad_MC(g, 0.0, 1.0, N[i])
    print("n = %.i, Quad MC= %.10f, blad = %.10f, wariancja = %.10f, [%.10f, %.10f]" % (N[i], Q, abs(np.pi-Q), VAR, L_CI, R_CI) )

n = 11, Quad MC= 2.4085126898, blad = 0.7330799638, wariancja = 0.5050131894, [1.9799789575, 2.8370464222]
n = 121, Quad MC= 2.6177453236, blad = 0.5238473299, wariancja = 1.0491650422, [2.4315112186, 2.8039794287]
n = 1331, Quad MC= 3.2863443178, blad = 0.1447516642, wariancja = 15.1187264992, [3.0731878214, 3.4995008142]
n = 14641, Quad MC= 3.0872382553, blad = 0.0543543983, wariancja = 11.2738761481, [3.0317397540, 3.1427367565]
n = 161051, Quad MC= 3.1367835229, blad = 0.0048091307, wariancja = 17.1451998220, [3.1161478062, 3.1574192397]
n = 19487171, Quad MC= 3.1415194475, blad = 0.0000732061, wariancja = 39.2828616345, [3.1386798476, 3.1443590474]
n = 214358881, Quad MC= 3.1420759374, blad = 0.0004832838, wariancja = 28.8226477509, [3.1413425622, 3.1428093125]
Wall time: 15.5 s


Zaimplementować najprostszą metodę Monte Carlo obliczania całki

$$I(f)=\int\limits_{-\infty}^{+\infty} f(x)\cdot\frac{1}{\sqrt{2\pi}}\cdot e^{-x^2/2} dx$$

Przedstawić implementację używającą modułu $\color{green}{numpy}$ oraz wykonać testy numeryczne.

Dla ustalonego $n\in\mathbb{N}$ metoda ta ma postać

$$MC_n(f) = \frac{1}{n}\sum\limits_{j=1}^n f(\xi_j), $$

gdzie $\xi_1,\xi_2,\ldots,\xi_n$ są niezależnymi zmiennymi losowymi o rozkładzie normalnym o średniej $0$ i odchyleniu standardowym $1$.

In [13]:
I2 = 0

In [14]:
def quad_MC_2(f, n):
    random_samples = np.random.normal(0,1,n).astype('float64')
    f_values = f(random_samples)
    
    return np.mean(f_values)

In [15]:
def f_1(x):
    return np.sin(x)

In [16]:
N = [11, 11**2, 11**3, 11**4, 11**6, 11**8]

In [17]:
%%time
for i in range(0, len(N)):
    Q = quad_MC_2(f_1, N[i])
    ERR = abs(I2-Q)
    print("n = %.i, wartość kwadratury Monte Carlo= %.10f, błąd = %.10f" % (N[i], Q, ERR) )

n = 11, wartość kwadratury Monte Carlo= 0.2429899532, błąd = 0.2429899532
n = 121, wartość kwadratury Monte Carlo= -0.0326790906, błąd = 0.0326790906
n = 1331, wartość kwadratury Monte Carlo= 0.0066497507, błąd = 0.0066497507
n = 14641, wartość kwadratury Monte Carlo= -0.0036848455, błąd = 0.0036848455
n = 1771561, wartość kwadratury Monte Carlo= 0.0007361078, błąd = 0.0007361078
n = 214358881, wartość kwadratury Monte Carlo= 0.0000768222, błąd = 0.0000768222
Wall time: 6.46 s


In [18]:
def f_2(x):
    return (1/(x**2+1))*np.sqrt(np.pi * 2) * np.exp((x**2)/2)

In [19]:
%%time
for i in range(0, len(N)):
    Q = quad_MC_2(f_2, N[i])
    ERR = abs(np.pi-Q)
    print("n = %.i, wartość kwadratury Monte Carlo= %.10f, błąd = %.10f" % (N[i], Q, ERR) )

n = 11, wartość kwadratury Monte Carlo= 2.6473580366, błąd = 0.4942346170
n = 121, wartość kwadratury Monte Carlo= 2.4271251165, błąd = 0.7144675371
n = 1331, wartość kwadratury Monte Carlo= 2.8985248433, błąd = 0.2430678103
n = 14641, wartość kwadratury Monte Carlo= 2.6203342410, błąd = 0.5212584126
n = 1771561, wartość kwadratury Monte Carlo= 2.8214811998, błąd = 0.3201114538
n = 214358881, wartość kwadratury Monte Carlo= 2.7927712001, błąd = 0.3488214535
Wall time: 8.68 s


Symulacja CDF rozkładu normalnego $N(0,1)$, tj.

$$\Phi(t)=\int\limits_{-\infty}^t \frac{1}{\sqrt{2\pi}}e^{-x^2/2}dx$$

In [20]:
def indicator(t, x):
    return float(x<=t)

In [21]:
def CDF_Normal(t, n):
    random_sample = np.random.normal(0,1,n)
    
    random_binary = np.where(random_sample <= t, 1.0, 0.0)
    
    return np.mean(random_binary)

In [22]:
CDF_Normal(0.0, 10**7)

0.4998713

Numerycznie sprawdzić następującą własność dystrybuanty $\Phi$ rozkładu normalnego $N(0,1)$:

$$\Phi(x)+\Phi(-x)=1, \quad\forall_{x\in\mathbb{R}}$$

In [23]:
CDF_Normal(0.23, 10**7)+CDF_Normal(-0.23, 10**7)

1.0001776

In [24]:
print(1.0-abs(CDF_Normal(0.23, 10**7)+CDF_Normal(-0.23, 10**7)))

-0.0001626999999999601


Porównanie z implementacjami biblioteki SciPy

In [25]:
import scipy.stats as sc_stats

In [26]:
dist = sc_stats.norm(0,1)

In [27]:
cdf = dist.cdf(0.0)
print(cdf)

0.5


In [28]:
print('%.17f' % float(dist.cdf(0.23)+dist.cdf(-0.23)))

1.00000000000000000


Dla dowolnego $K\geq 0$ zaimplementować metodę Monte Carlo obliczania całki

$$I(f)=\int\limits_{-\infty}^{+\infty} (x-K)^{+}\cdot\frac{1}{\sqrt{2\pi}}\cdot e^{-x^2/2} dx$$

In [29]:
def call_option(K, n):
    random_sample_0 = np.random.normal(0,1,n)
    
    random_sample_1 = np.where(random_sample_0 >= K, random_sample_0-K, 0.0)
    
    return np.mean(random_sample_1)

In [30]:
call_option(3, 10**7)

0.00038675690790802007